# NIST SRM 731
### Date:  2019/06/27
### Author:  Donald Windover

## Functions for using the third-order splines provided in the Certificate of Analysis

#####  Note:  This notebooks is provided as a means to calculate expansivity and expansion values for SRM 731, based on the third-order polynomial provided for Temperature values between 293 K and 640 K.  This notebook in no way supersedes the validity of the original Certificate of Analysis, and further, provided no more information, than is present within the original document.

### The NIST SRM 731 Certificate of Analysis can be found at:

https://www-s.nist.gov/srmors/certificates/731L1.pdf

#### Python packages loading:

Here we import the numpy, scipy, and scipy.integrate packages used in calculations:

```python
import numpy as np
import scipy as sp
import scipy.integrate as integrate
from IPython.display import display, Math
```

For numpy and scipy documentation, see:

https://docs.scipy.org/doc/



In [1]:
import numpy as np
import scipy as sp
import scipy.integrate as integrate

### Here we implement the 293<= T (in K) <= 640 third-order spline polynomial from the COA

\begin{equation}
\alpha = 0.8651 + 2.3569 \times 10 ^{-2} \dot T - 4.2277 \times 10 ^{-5} \dot T^2 + 2.5408 \times 10^{-8} \dot T^3
\end{equation}

In [2]:
def expansivity_high(Temp):
    expansivity = (0.8651 + (2.3569e-2 * Temp) - (4.2277e-5 * Temp**2) + (2.5408e-8 * Temp**3))
    return expansivity

### We can reproduce the Expansivity values for the table on page 2 from the COA
#### first, by putting in the temperatures in the table into an array

In [100]:
print('Temps (in K)')
Temp_chart = np.array([293, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500, 520, 540, 560, 580, 600, 620, 640, 660, 680],dtype=float)
for i in np.arange(len(Temp_chart)):
    print(Temp_chart[i])

Temps (in K)
293.0
300.0
320.0
340.0
360.0
380.0
400.0
420.0
440.0
460.0
480.0
500.0
520.0
540.0
560.0
580.0
600.0
620.0
640.0
660.0
680.0


### and then, by using our third order polynomial function above to calculate expansivity at each temp, and round to two decimals...

#### Breakdown of the python statement below:

1. the first print statement provides the row information
2. the for statement runs through each temperature
3. the Expansivity is pulled from ln 2 and is calculated for each temperature, rounded to 2 decimal places
4. the temperature and Expansivity are then printed

In [101]:
print("Temp (in K) | Expansivity ( x 10-6) ")
for i in np.arange(len(Temp_chart)):
    Expansivity = np.round(expansivity_high(Temp_chart[i]), decimals = 2)
    print(Temp_chart[i],"        | ", Expansivity)

Temp (in K) | Expansivity ( x 10-6) 
293.0         |  4.78
300.0         |  4.82
320.0         |  4.91
340.0         |  4.99
360.0         |  5.06
380.0         |  5.11
400.0         |  5.15
420.0         |  5.19
440.0         |  5.21
460.0         |  5.23
480.0         |  5.25
500.0         |  5.26
520.0         |  5.26
540.0         |  5.27
560.0         |  5.27
580.0         |  5.27
600.0         |  5.27
620.0         |  5.28
640.0         |  5.29
660.0         |  5.31
680.0         |  5.33


### We need to integrate the expansivity... Here is my personal view of integration


![alt text][integration]

[integration]: https://imgs.xkcd.com/comics/differentiation_and_integration.png "https://imgs.xkcd.com/comics/differentiation_and_integration.png"

https://imgs.xkcd.com/comics/differentiation_and_integration.png

### I chose to perform a simple numerical integration using scipy.
see:  https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html

We are using the relation between Expansivity and Expansion from the COA

\begin{equation}
\alpha \dot dT = {{1} \over {L_{293}}}dL
\end{equation}

\begin{equation}
\int_{293}^T \alpha \dot dT = \int_{L_{293}}^L {{1}\over{L_{293}}} dL = {{L - L_{293}} \over {L_{293}}}
\end{equation}


Breakdown of the python statement below:

1. the first print statement provides the row information
2. the for statement runs through each temperature
3. the Expansivity is numerically integrated between 293 and Temp_chart value from the for-loop step to provide expansion
4. the Expansion is rounded to 0 decimal places
5. the Temperature and Expansion are printed

In [102]:
print('Temp (in K) | Expansion[ \u0394 L/ L_293] x10-6')
for i in np.arange(len(Temp_chart)):
    Expansion = integrate.quad (lambda x: expansivity_high(x), Temp_chart[0],Temp_chart[i])
    Expansion = np.round(Expansion[0], decimals = 0)
    print(Temp_chart[i],"        | ", Expansion)

Temp (in K) | Expansion[ Δ L/ L_293] x10-6
293.0         |  0.0
300.0         |  34.0
320.0         |  131.0
340.0         |  230.0
360.0         |  330.0
380.0         |  432.0
400.0         |  535.0
420.0         |  638.0
440.0         |  742.0
460.0         |  847.0
480.0         |  952.0
500.0         |  1057.0
520.0         |  1162.0
540.0         |  1267.0
560.0         |  1372.0
580.0         |  1478.0
600.0         |  1583.0
620.0         |  1689.0
640.0         |  1795.0
660.0         |  1901.0
680.0         |  2007.0


### But since Expansivity is modeled as a polynomial, we can just do integration on each polynomial term

#### increasing the power by one and dividing by the new power 


\begin{equation}
\int \alpha \dot dT = \int 0.8651 + 2.3569 \times 10 ^{-2} \dot T - 4.2277 \times 10 ^{-5} \dot T^2 + 2.5408 \times 10^{-8} \dot T^3 \dot dT
\end{equation}


\begin{equation}
\int \alpha \dot dT = 0.8651 \dot (T) + 2.3569 \times 10 ^{-2} \dot (T^2/2) - 4.2277 \times 10 ^{-5} \dot (T^3/3) + 2.5408 \times 10^{-8} \dot (T^4/4)
\end{equation}

In [103]:
def expansion_high(Temp):
    const_a = 0.8651
    const_b = 2.3569e-2
    const_c = -4.2277e-5
    const_d = 2.5408e-8
    expansion =  const_a*Temp + const_b * Temp**2/2 + const_c *Temp**3/3 + const_d * Temp**4/4
    return expansion

In [104]:
print('Temp (in K) | Expansion[\u0394 L/ L293]  x10-6 ')
for i in np.arange(len(Temp_chart)):
    #Expansion = integrate.quad (lambda x: expansivity_high(x), Temp_chart[0],Temp_chart[i])
    Expansion = expansion_high(Temp_chart[i]) - expansion_high(Temp_chart[0])
    Expansion = np.round(Expansion, decimals = 0)
    print(Temp_chart[i],'        | ', Expansion)

Temp (in K) | Expansion[Δ L/ L293]  x10-6 
293.0         |  0.0
300.0         |  34.0
320.0         |  131.0
340.0         |  230.0
360.0         |  330.0
380.0         |  432.0
400.0         |  535.0
420.0         |  638.0
440.0         |  742.0
460.0         |  847.0
480.0         |  952.0
500.0         |  1057.0
520.0         |  1162.0
540.0         |  1267.0
560.0         |  1372.0
580.0         |  1478.0
600.0         |  1583.0
620.0         |  1689.0
640.0         |  1795.0
660.0         |  1901.0
680.0         |  2007.0


### Both methods of integration match in results!

#### we see a slight variation of 1ppm at 640 and 660 from tabulated values, but I am guessing we are seeing a rounding error from calculations performed in the 1970s.

### Now, lets check the values you needed calculated....

#### We make a new table of temperatures in $^{\circ}$C, and then convert them to K

````python
Temp_of_interest_in_C = [30, 50, 80, 300]
Temp_of_interest_in_K = Temp_of_interest_in_C + 273.15
````

In [8]:
Temp_of_interest_in_C = [30, 50, 80, 300]
Temp_of_interest_in_K = np.array(Temp_of_interest_in_C) + 273.15
print("Temperature (in K) = ", Temp_of_interest_in_K)

Temperature (in K) =  [ 303.15  323.15  353.15  573.15]


#### We then run the expansivity calculations between 300 $^{\circ}$C and any requested temperatures.

Breakdown of the python statement below:

1. the first print statement provides the row information
2. the for statement runs through each temperature
3. the Expansivity is called for each temp
4. the Expansivity is rounded to 2 decimal places
5. the Temperature and Expansivity are printed

In [9]:
print("Temperature (in K) | Expansivity [Temperature] x 10-6 ")
for i in np.arange(len(Temp_of_interest_in_K)):
    Expansivity = expansivity_high(Temp_of_interest_in_K[i])
    Expansivity = np.round(Expansivity, decimals = 2)
    print(Temp_of_interest_in_K[i],  '            | ', Expansivity)

Temperature (in K) | Expansivity [Temperature] x 10-6 
303.15             |  4.83
323.15             |  4.92
353.15             |  5.03
573.15             |  5.27


#### We then run the expansion calculations between 300 $^{\circ}$C and the lower values requested

Breakdown of the python statement below:

1. the first print statement provides the row information
2. the for statement runs through each temperature
3. the Expansion is called for each temp, and subtracted from expansion at 300 C
4. the Expansion is rounded to 0 decimal places
5. the Temperature and Expansion are printed

In [10]:
print("Expansion(573.15 (in K)) - Expansion(T (in K)) | Expansion[\u0394 L(Temperature)/ L\u2085\u2087\u2083] x 10\u207B\u2076 ")
for i in np.arange(len(Temp_of_interest_in_K)):
    Expansion = expansion_high(Temp_of_interest_in_K[-1]) - expansion_high(Temp_of_interest_in_K[i])
    Expansion = np.round(Expansion, decimals = 0)
    print("Expansion (", Temp_of_interest_in_K[-1], ") - Expansion (", Temp_of_interest_in_K[i], ')    | ', Expansion)

Expansion(573.15 (in K)) - Expansion(T (in K)) | Expansion[Δ L(Temperature)/ L₅₇₃] x 10⁻⁶ 
Expansion ( 573.15 ) - Expansion ( 303.15 )    |  1393.0
Expansion ( 573.15 ) - Expansion ( 323.15 )    |  1295.0
Expansion ( 573.15 ) - Expansion ( 353.15 )    |  1146.0
Expansion ( 573.15 ) - Expansion ( 573.15 )    |  0.0


#### Feel free to change the temperatures in the "Temp_of_interest" section, in order to calculate expansions and expansivities for values not provided within the table (ONLY valid for T between 293 K and 640 K).